# Minesweeper

## 1. Create a Minesweeper

In [1]:
import numpy as np

In [2]:
class Minesweeper(object):
    '''
    size:           (2-element tuple) size of mine map and game board
    n_mine:         (int) number of mines
    n_remaining:    (int) number of remaining mines
    game_status:    (int) game status:
                        0: ongoing
                        1: win
                        2: lose
    board:          (2D list, size) player game board:
                         : not visited
                        n(0-8): visited with n mines around
                        P: flag as mine
                        #: explosion
    __n_unvisited:  (int) number of unvisited cells
    __map:          (2D ndarray, size) map of mines: 
                        0: cell without mine
                        1: cell with mine
    __mine_count:   (2D ndarray, size) count of mines in the 8 cells around
    '''

    # initialization
    def __init__(self, size=(9,9), n_mine=10):
        self.size = size
        self.n_mine = n_mine
        self.n_remaining = n_mine
        self.game_status = 0
        
        self.__n_unvisited = size[0]*size[1]

        # if n_mine > size, return error
        if n_mine > size[0]*size[1]:
            print("Error: too many mines")
            return
        
        # randomly pick cells for mines
        self.__mines = np.random.choice(size[0]*size[1], n_mine, False)
        
        # generate mine map (size[0]*size[1]; 1: cell with mine, 0: cell without mine)
        self.__map = np.zeros(size, dtype=np.int8)
        for mine in self.__mines:
            self.__map[mine//size[1]][mine%size[1]] = 1

        # generate mine count (size[0]*size[1]; number of mines in 8 cells around)
        self.__mine_count = np.zeros(size, dtype=np.int8)
        for mine in self.__mines:
            i = mine//size[1]
            j = mine%size[1]
            if i>0 and j>0:
                self.__mine_count[i-1][j-1] += 1
            if j>0:
                self.__mine_count[i][j-1] += 1
            if i<size[0]-1 and j>0:
                self.__mine_count[i+1][j-1] += 1
            if i>0: 
                self.__mine_count[i-1][j] += 1
            if i<size[0]-1:
                self.__mine_count[i+1][j] += 1
            if i>0 and j<size[1]-1:
                self.__mine_count[i-1][j+1] += 1
            if j<size[1]-1:
                self.__mine_count[i][j+1] += 1
            if i<size[0]-1 and j<size[1]-1:
                self.__mine_count[i+1][j+1] += 1

        # generate game board (size[0]*size[1])
        self.board = [[" " for i in range(size[1])] for i in range(size[0])]

        # show game board
        self.__show_board()
        

    # private methods: 
    # show game board
    def __show_board(self):
        print("remaining mines: ", end="")
        print(self.n_remaining)
        for j in range(self.size[1]):
            print("----", end="")
        print("-")
        for i in range(self.size[0]):
            print("|", end="")
            for j in range(self.size[1]):
                print(" ", end="")
                print(self.board[i][j], end="")
                print(" |", end="")
            print("")
            for j in range(self.size[1]):
                print("----", end="")
            print("-")
        print("\n")

    # check flags
    def __check_board(self):
        for mine in self.__mines:
            i = mine//self.size[1]
            j = mine%self.size[1]
            if self.board[i][j]!="P":
                self.board[i][j] = "#"
                self.game_status = 2
                print("you lose")
                return
        self.game_status = 1
        print("you win")

    # visit cell quietly
    def __visit_cell(self, cell):
        if len(cell)!=2 or cell[0] not in range(self.size[0]) or cell[1] not in range(self.size[1]):
            print("invalid cell index")
            return
        
        if self.__map[cell[0]][cell[1]] == 1:
            self.board[cell[0]][cell[1]] = "#"
            self.game_status = 2
            print("you lose")
            return
        
        if self.board[cell[0]][cell[1]] != " ":
            return
        
        self.board[cell[0]][cell[1]] = str(self.__mine_count[cell[0]][cell[1]])
        self.__n_unvisited -= 1

        if self.__n_unvisited == 0:
            self.__check_board()

        # iterate if no mine detected
        i = cell[0]
        j = cell[1]
        if self.__mine_count[i][j] == 0: 
            if i>0 and j>0:
                self.__visit_cell((i-1,j-1))
            if j>0:
                self.__visit_cell((i,j-1))
            if i<self.size[0]-1 and j>0:
                self.__visit_cell((i+1,j-1))
            if i>0: 
                self.__visit_cell((i-1,j))
            if i<self.size[0]-1:
                self.__visit_cell((i+1,j))
            if i>0 and j<self.size[1]-1:
                self.__visit_cell((i-1,j+1))
            if j<self.size[1]-1:
                self.__visit_cell((i,j+1))
            if i<self.size[0]-1 and j<self.size[1]-1:
                self.__visit_cell((i+1,j+1))        

    # flag cell quietly
    def __flag_cell(self, cell):
        if len(cell)!=2 or cell[0] not in range(self.size[0]) or cell[1] not in range(self.size[1]):
            print("invalid cell index")
            return
        if self.n_remaining == 0:
            print("no flag remaining; please check existing flags")
            return
        
        if self.board[cell[0]][cell[1]] == " ":
            self.board[cell[0]][cell[1]] = "P"
            self.n_remaining -= 1
            self.__n_unvisited -= 1
        elif self.board[cell[0]][cell[1]] == "P":
            self.board[cell[0]][cell[1]] = " "
            self.n_remaining += 1
            self.__n_unvisited += 1

        if self.__n_unvisited == 0:
            self.__check_board()


    # public methods: 
    # player actions: visit, flag
    def visit(self, cell):
        print("visit", end="")
        print(cell)
        self.__visit_cell(cell)
        self.__show_board()
        
    def flag(self, cell):
        print("flag", end="")
        print(cell)
        self.__flag_cell(cell)
        self.__show_board()


## 2. Game Rules and Actions

#### Start a new game and display the game board
- Change the random seed to generate different games
- The game board is divided into cells with several mines distributed, each cell with 0 or 1 mine
- Your goal is to flag the cells that contain mines without triggering the mines
- "Remaining mines" on the upper side of the game board shows the number of mines you will need to find out and flag in order to win the game

In [9]:
np.random.seed(1)
game1 = Minesweeper()

remaining mines: 10
-------------------------------------
|   |   |   |   |   |   |   |   |   |
-------------------------------------
|   |   |   |   |   |   |   |   |   |
-------------------------------------
|   |   |   |   |   |   |   |   |   |
-------------------------------------
|   |   |   |   |   |   |   |   |   |
-------------------------------------
|   |   |   |   |   |   |   |   |   |
-------------------------------------
|   |   |   |   |   |   |   |   |   |
-------------------------------------
|   |   |   |   |   |   |   |   |   |
-------------------------------------
|   |   |   |   |   |   |   |   |   |
-------------------------------------
|   |   |   |   |   |   |   |   |   |
-------------------------------------




AttributeError: 'Minesweeper' object has no attribute '__map'

#### Visit a cell
- If you visit a cell with a mine, the mine will explode (a "#" shown in the cell) and you lose the game immediately
- If you visit a safe cell without a mine, a number will display showing the total number of mines in the 8 adjacent cells
- Nearby cells will be automatically visited if no mines are detected (a "0" shown in the cell) in the 8 adjacent cells

In [4]:
game1.visit((0,0))

visit(0, 0)
remaining mines: 10
-------------------------------------
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------
| 1 | 1 | 0 | 1 | 1 | 1 | 0 | 0 | 0 |
-------------------------------------
|   | 1 | 1 | 2 |   | 1 | 0 | 0 | 0 |
-------------------------------------
|   |   |   |   | 2 | 1 | 0 | 1 | 1 |
-------------------------------------
|   |   |   | 2 | 1 | 0 | 0 | 1 |   |
-------------------------------------
|   |   |   | 1 | 0 | 1 | 1 | 2 |   |
-------------------------------------
|   |   |   | 2 | 1 | 2 |   |   |   |
-------------------------------------
|   |   |   |   |   |   |   |   |   |
-------------------------------------




In [5]:
game1.visit((3,0))

visit(3, 0)
you lose
remaining mines: 10
-------------------------------------
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------
| 1 | 1 | 0 | 1 | 1 | 1 | 0 | 0 | 0 |
-------------------------------------
| # | 1 | 1 | 2 |   | 1 | 0 | 0 | 0 |
-------------------------------------
|   |   |   |   | 2 | 1 | 0 | 1 | 1 |
-------------------------------------
|   |   |   | 2 | 1 | 0 | 0 | 1 |   |
-------------------------------------
|   |   |   | 1 | 0 | 1 | 1 | 2 |   |
-------------------------------------
|   |   |   | 2 | 1 | 2 |   |   |   |
-------------------------------------
|   |   |   |   |   |   |   |   |   |
-------------------------------------




#### Flag a cell
- Flag an unvisited cell (a "P" shown in the cell) to label a possible mine
- "Remaining mines" will decrease by 1 once you flag a cell (whether correctly or not)

In [6]:
np.random.seed(1)
game1 = Minesweeper()
game1.visit((0,0))
game1.flag((3,0))

remaining mines: 10
-------------------------------------
|   |   |   |   |   |   |   |   |   |
-------------------------------------
|   |   |   |   |   |   |   |   |   |
-------------------------------------
|   |   |   |   |   |   |   |   |   |
-------------------------------------
|   |   |   |   |   |   |   |   |   |
-------------------------------------
|   |   |   |   |   |   |   |   |   |
-------------------------------------
|   |   |   |   |   |   |   |   |   |
-------------------------------------
|   |   |   |   |   |   |   |   |   |
-------------------------------------
|   |   |   |   |   |   |   |   |   |
-------------------------------------
|   |   |   |   |   |   |   |   |   |
-------------------------------------


visit(0, 0)
remaining mines: 10
-------------------------------------
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------
| 1 | 1 | 0 | 1 | 1 | 1 | 0 | 0 | 

- Flag a flagged cell (a "P" shown in the cell) to cancel the flag

In [7]:
game1.flag((3,0))

flag(3, 0)
remaining mines: 10
-------------------------------------
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------
| 1 | 1 | 0 | 1 | 1 | 1 | 0 | 0 | 0 |
-------------------------------------
|   | 1 | 1 | 2 |   | 1 | 0 | 0 | 0 |
-------------------------------------
|   |   |   |   | 2 | 1 | 0 | 1 | 1 |
-------------------------------------
|   |   |   | 2 | 1 | 0 | 0 | 1 |   |
-------------------------------------
|   |   |   | 1 | 0 | 1 | 1 | 2 |   |
-------------------------------------
|   |   |   | 2 | 1 | 2 |   |   |   |
-------------------------------------
|   |   |   |   |   |   |   |   |   |
-------------------------------------




#### Win the game
You win the game when the following 2 conditions are met:
- Each cell is either flagged or visited
- All mines are correctly flagged

## 3. Let's Begin!

In [8]:
np.random.seed(1)
game1 = Minesweeper()
game1.visit((0,0))
game1.flag((3,0))
game1.flag((3,4))
game1.visit((4,0))
game1.visit((4,1))
game1.visit((4,2))
game1.flag((4,3))
game1.flag((5,1))
game1.flag((5,2))
game1.flag((5,8))
game1.visit((5,0))
game1.visit((6,0))
game1.visit((6,1))
game1.visit((6,2))
game1.visit((6,8))
game1.visit((7,2))
game1.visit((7,8))
game1.flag((7,6))
game1.visit((8,6))
game1.visit((8,5))
game1.flag((8,4))
game1.visit((8,3))
game1.flag((8,2))
game1.flag((7,0))
game1.visit((7,1))
game1.visit((8,0))
game1.visit((8,1))

remaining mines: 10
-------------------------------------
|   |   |   |   |   |   |   |   |   |
-------------------------------------
|   |   |   |   |   |   |   |   |   |
-------------------------------------
|   |   |   |   |   |   |   |   |   |
-------------------------------------
|   |   |   |   |   |   |   |   |   |
-------------------------------------
|   |   |   |   |   |   |   |   |   |
-------------------------------------
|   |   |   |   |   |   |   |   |   |
-------------------------------------
|   |   |   |   |   |   |   |   |   |
-------------------------------------
|   |   |   |   |   |   |   |   |   |
-------------------------------------
|   |   |   |   |   |   |   |   |   |
-------------------------------------


visit(0, 0)
remaining mines: 10
-------------------------------------
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------
| 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------
| 1 | 1 | 0 | 1 | 1 | 1 | 0 | 0 | 